# Implement Bert

Author: Crystal

Currently rate is 40,000 abstracts per hour. 

In [1]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
#nltk.download("punkt")

from nltk import tokenize


import statistics #calculate mean and others

In [21]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') # this model provides quick model with high quality
#embedder = SentenceTransformer('paraphrase-mpnet-base-v2') # this model provides the best quality

abstracts = pd.read_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/smaller-final-dataset.pkl")
len(abstracts)

690814

In [22]:
#abstracts = abstracts.iloc[0:40000] #1.a
#abstracts = abstracts.iloc[40001:200001] #1.b
#abstracts = abstracts.iloc[80001:140000] #1.c
#abstracts = abstracts.iloc[140001:200000] #1.d


#abstracts = abstracts.iloc[200001:300000] #3
#abstracts = abstracts.iloc[300001:400000] #4
#abstracts = abstracts.iloc[400001:530000] #5

#abstracts = abstracts.iloc[530001:690814] #6

# Functions

In [23]:
def get_corpus_embeddings(dir):
    with open(dir) as f:
        ai_text = f.read()
    ai_corpus = tokenize.sent_tokenize(ai_text) #sentence tokenization
    ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=True) # embeddings
    return ai_embeddings


# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [24]:
import datetime
start_time = datetime.datetime.now()
print("Start time for calculating embeddings:", start_time)

ai_embeddings = get_corpus_embeddings("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/ai_wiki_text.txt")

abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts), "of", "embedding score at", end_time)
print("It took", end_time-start_time, "to run.")


Start time for calculating embeddings: 2021-07-14 13:17:19.823037


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Finished calculating  129999 of embedding score at 2021-07-14 16:28:19.929866
It took 3:11:00.106829 to run.


In [25]:
start_time = datetime.datetime.now()
print("Start time for calculating average score:", start_time)


sentence_score= []
for abstract in abstracts["score"]:
    sentence_score.append([statistics.mean(i) for i in abstract])

abstracts["sentence_score"]=sentence_score
abstracts

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts), "of", "average score at", end_time)
print("It took", end_time-start_time, "to run.")

Start time for calculating average score: 2021-07-14 16:28:20.015391
Finished calculating  129999 of average score at 2021-07-14 16:28:58.215902
It took 0:00:38.200511 to run.


In [26]:
abstracts.to_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/abstracts_embedding_score_5.csv', index = False)   